# $GPy$*Torch*

In [2]:
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
import matplotlib
from matplotlib import pyplot as plt
import math
import numpy as np
import torch as tc
import gpytorch as gpy

#configure plotting
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
%config InlineBackend.figure_format = 'retina'
%config IPCompleter.greedy = True
%config IPCompleter.use_jedi = True

InteractiveShell.ast_node_interactivity = 'all'

matplotlib.rcParams['figure.figsize'] = (16,9)
matplotlib.rcParams['font.size'] = 24
matplotlib.rcParams['font.family'] = 'serif'

np.random.seed(1)

For most GP regression models, you will need to construct the following GPyTorch objects:

* A **GP Model** (`gpytorch.models.ExactGP`) - This handles most of the inference.
* A **Likelihood** (`gpytorch.likelihoods.GaussianLikelihood`) - This is the most common likelihood used for GP regression.
* A **Mean** - This defines the prior mean of the GP.

    * If you don’t know which mean to use, a `gpytorch.means.ConstantMean()` is a good place to start.

* A **Kernel** - This defines the prior covariance of the GP.

    * If you don’t know which kernel to use, a `gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())` is a good place to start.

* A **MultivariateNormal Distribution** (`gpytorch.distributions.MultivariateNormal`) - This is the object used to represent multivariate normal distributions.

**The GP Model**

* For setting up the model, we need to implement a `__init__` that can take training 
data and a likelihood, and construct for the `forward` method. 
* The `forward` method takes some $nxd$ data $X$ and returns a **MultiVariateNormal** with prior mean and covariance evaluated at $X$. i.e. a vector $mu(x)$ and the $nxn$ matrix $K_{xx}$ representing the prior mean and covariance matrices of the GP.

* E.g. , Adding kernel modulesis valid in both these ways.

    ```python
    self.covar_module = ScaleKernel(RBFKernel() + WhiteNoiseKernel())
    covar_x = self.rbf_kernel_module(x) + self.white_noise_module(x)

    ```
    
* Example:
    
```python
        
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
```

**Model modes**

* Like most PyTorch modules, the `ExactGP` has a `.train()` and `.eval()` mode. - `.train()` mode is for optimizing model hyperameters. - `.eval()` mode is for computing predictions through the model posterior.

**Training the model**

In the next cell, we handle using Type-II MLE to train the hyperparameters of the Gaussian process.

The most obvious difference here compared to many other GP implementations is that, as in standard PyTorch, the core training loop is written by the user. In GPyTorch, we make use of the standard PyTorch optimizers as from `torch.optim`, and all trainable parameters of the model should be of type `torch.nn.Parameter`. Because GP models directly extend `torch.nn.Module`, calls to methods like `model.parameters()` or `model.named_parameters()` function as you might expect coming from PyTorch.

In most cases, the boilerplate code below will work well. It has the same basic components as the standard PyTorch training loop:

    1. Zero all parameter gradients
    2. Call the model and compute the loss
    3. Call backward on the loss to fill in gradients
    4. Take a step on the optimizer

However, defining custom training loops allows for greater flexibility. For example, it is easy to save the parameters at each step of training, or use different learning rates for different parameters (which may be useful in deep kernel learning for example).

```python
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([{'params': model.parameters()},], lr=0.1)
# Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 50
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f log_lengthscale: %.3f log_noise: %.3f' %
         (
             i + 1, training_iter, loss.item(),
             model.covar_module.base_kernel.log_lengthscale.item(),
             model.likelihood.log_noise.item()
         ))
    optimizer.step()
```
```output
Iter 1/50 - Loss: 1.084   log_lengthscale: 0.000   log_noise: 0.000
Iter 2/50 - Loss: 1.043   log_lengthscale: -0.100   log_noise: -0.100
Iter 3/50 - Loss: 1.004   log_lengthscale: -0.196   log_noise: -0.200
Iter 4/50 - Loss: 0.964   log_lengthscale: -0.293   log_noise: -0.300
Iter 5/50 - Loss: 0.922   log_lengthscale: -0.387   log_noise: -0.399
Iter 6/50 - Loss: 0.877   log_lengthscale: -0.479   log_noise: -0.499
Iter 7/50 - Loss: 0.825   log_lengthscale: -0.572   log_noise: -0.598
Iter 8/50 - Loss: 0.767   log_lengthscale: -0.667   log_noise: -0.698
Iter 9/50 - Loss: 0.705   log_lengthscale: -0.762   log_noise: -0.799
Iter 10/50 - Loss: 0.644   log_lengthscale: -0.860   log_noise: -0.899
Iter 11/50 - Loss: 0.590   log_lengthscale: -0.960   log_noise: -1.001
Iter 12/50 - Loss: 0.543   log_lengthscale: -1.058   log_noise: -1.102
Iter 13/50 - Loss: 0.502   log_lengthscale: -1.150   log_noise: -1.204
Iter 14/50 - Loss: 0.462   log_lengthscale: -1.234   log_noise: -1.306
Iter 15/50 - Loss: 0.426   log_lengthscale: -1.303   log_noise: -1.408
Iter 16/50 - Loss: 0.389   log_lengthscale: -1.360   log_noise: -1.509
Iter 17/50 - Loss: 0.360   log_lengthscale: -1.404   log_noise: -1.611
Iter 18/50 - Loss: 0.321   log_lengthscale: -1.432   log_noise: -1.712
Iter 19/50 - Loss: 0.280   log_lengthscale: -1.454   log_noise: -1.812
Iter 20/50 - Loss: 0.250   log_lengthscale: -1.465   log_noise: -1.911
Iter 21/50 - Loss: 0.227   log_lengthscale: -1.469   log_noise: -2.010
Iter 22/50 - Loss: 0.188   log_lengthscale: -1.461   log_noise: -2.108
Iter 23/50 - Loss: 0.158   log_lengthscale: -1.442   log_noise: -2.204
Iter 24/50 - Loss: 0.125   log_lengthscale: -1.411   log_noise: -2.300
Iter 25/50 - Loss: 0.095   log_lengthscale: -1.377   log_noise: -2.393
Iter 26/50 - Loss: 0.070   log_lengthscale: -1.340   log_noise: -2.485
Iter 27/50 - Loss: 0.050   log_lengthscale: -1.298   log_noise: -2.574
Iter 28/50 - Loss: 0.032   log_lengthscale: -1.256   log_noise: -2.662
Iter 29/50 - Loss: 0.014   log_lengthscale: -1.218   log_noise: -2.746
Iter 30/50 - Loss: 0.003   log_lengthscale: -1.182   log_noise: -2.828
Iter 31/50 - Loss: -0.001   log_lengthscale: -1.148   log_noise: -2.906
Iter 32/50 - Loss: -0.008   log_lengthscale: -1.121   log_noise: -2.980
Iter 33/50 - Loss: -0.012   log_lengthscale: -1.102   log_noise: -3.049
Iter 34/50 - Loss: -0.011   log_lengthscale: -1.103   log_noise: -3.114
Iter 35/50 - Loss: -0.014   log_lengthscale: -1.114   log_noise: -3.174
Iter 36/50 - Loss: -0.014   log_lengthscale: -1.138   log_noise: -3.228
Iter 37/50 - Loss: -0.010   log_lengthscale: -1.169   log_noise: -3.275
Iter 38/50 - Loss: -0.011   log_lengthscale: -1.204   log_noise: -3.317
Iter 39/50 - Loss: -0.008   log_lengthscale: -1.239   log_noise: -3.352
Iter 40/50 - Loss: -0.001   log_lengthscale: -1.270   log_noise: -3.380
Iter 41/50 - Loss: -0.005   log_lengthscale: -1.296   log_noise: -3.401
Iter 42/50 - Loss: 0.008   log_lengthscale: -1.317   log_noise: -3.415
Iter 43/50 - Loss: 0.001   log_lengthscale: -1.331   log_noise: -3.422
Iter 44/50 - Loss: 0.009   log_lengthscale: -1.343   log_noise: -3.423
Iter 45/50 - Loss: 0.001   log_lengthscale: -1.350   log_noise: -3.419
Iter 46/50 - Loss: -0.001   log_lengthscale: -1.360   log_noise: -3.410
Iter 47/50 - Loss: 0.007   log_lengthscale: -1.374   log_noise: -3.397
Iter 48/50 - Loss: 0.000   log_lengthscale: -1.388   log_noise: -3.380
Iter 49/50 - Loss: -0.010   log_lengthscale: -1.396   log_noise: -3.359
Iter 50/50 - Loss: -0.008   log_lengthscale: -1.404   log_noise: -3.337
```

**Make predictions with the model**

* In the next cell, we make predictions with the model. To do this, we simply put the model and likelihood in `.eval()` mode, and call both modules on the test data.

* Just as a user defined GP model returns a `**MultivariateNormal**` containing the prior mean and covariance from `forward`, a trained GP model in `eval` mode returns a `**MultivariateNormal**` containing the posterior mean and covariance. Thus, getting the predictive mean and variance, and then sampling functions from the GP at the given test points could be accomplished with calls like:

```python
f_preds = model(test_x)
y_preds = likelihood(model(test_x))

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
f_samples = f_preds.sample(sample_shape=torch.Size(1000,))
```

*The `gpytorch.settings.fast_pred_var` context is not needed, but here we are giving a preview of using one of our cool features, getting faster predictive distributions using [LOVE](https://arxiv.org/abs/1803.06058).

```python
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0, 1, 51)
    observed_pred = likelihood(model(test_x))
```

**Plot the model fit**

In the next cell, we plot the mean and confidence region of the Gaussian process model. The `confidence_region` method is a helper method that returns 2 standard deviations above and below the mean.

```python
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
```
![Plot](https://gpytorch.readthedocs.io/en/latest/_images/examples_01_Simple_GP_Regression_Simple_GP_Regression_12_0.png)

## Classification

For most variational GP models, you will need to construct the following GPyTorch objects:

* A **GP Model** (`gpytorch.models.AbstractVariationalGP`) - This handles basic variational inference.
* A **Variational distribution** (`gpytorch.variational.VariationalDistribution`) - This tells us what form the variational distribution $q(u)$ should take.
* A **Variational strategy** (`gpytorch.variational.VariationalStrategy`) - This tells us how to transform a distribution $q(u)$ over the inducing point values to a distribution $q(f)$ over the latent function values for some input $x$.
* A **Likelihood** (`gpytorch.likelihoods.BernoulliLikelihood`) - This is a good likelihood for *binary classification*
* A **Mean** - This defines the prior mean of the GP.

    * If you don’t know which mean to use, a `gpytorch.means.ConstantMean()` is a good place to start.

* A **Kernel** - This defines the prior covariance of the GP.

    * If you don’t know which kernel to use, use a `gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())` is a good place to start.

* A **MultivariateNormal Distribution** (`gpytorch.distributions.MultivariateNormal`) - This

**The GP Model**

The `AbstractVariationalGP` model is GPyTorch’s simplist approximate inference model. It approximates the true posterior with a distribution specified by a `VariationalDistribution`, which is most commonly some form of `MultivariateNormal` distribution. The model defines all the variational parameters that are needed, and keeps all of this information under the hood.

The components of a user built `AbstractVariationalGP` model in GPyTorch are:

    1. An `__init__` method that constructs a mean module, a kernel module, a variational distribution object and a variational strategy object. This method should also be responsible for construting whatever other modules might be necessary.
    2. A forward method that takes in some $n×d$ data $x$ and returns a `MultivariateNormal` with the prior mean and covariance evaluated at $x$. In other words, we return the vector $μ(x)$ and the $n×n$ matrix $K_{xx}$ representing the prior mean and covariance matrix of the GP.

(For those who are unfamiliar with GP classification: even though we are performing classification, the GP model still returns a `MultivariateNormal`. The likelihood transforms this latent Gaussian variable into a Bernoulli variable)

Here we present a simple classification model, but it is possible to construct more complex models. See some of the scalable classification examples or deep kernel learning examples for some other examples.

```python
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy


class GPClassificationModel(AbstractVariationalGP):
    def __init__(self, train_x):
        variational_distribution = CholeskyVariationalDistribution(train_x.size(0))
        variational_strategy = VariationalStrategy(self, train_x, variational_distribution)
        super(GPClassificationModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred


# Initialize model and likelihood
model = GPClassificationModel(train_x)
likelihood = gpytorch.likelihoods.BernoulliLikelihood()
```

**Model Modes**

Like most PyTorch modules, the `ExactGP` has a `.train()` and `.eval()` mode. - `.train()` mode is for optimizing variational parameters model hyperameters. - `.eval()` mode is for computing predictions through the model posterior.

**Learn the variational parameters (and other hyperparameters)**

In the next cell, we optimize the variational parameters of our Gaussian process. In addition, this optimization loop also performs Type-II MLE to train the hyperparameters of the Gaussian process.

The most obvious difference here compared to many other GP implementations is that, as in standard PyTorch, the core training loop is written by the user. In GPyTorch, we make use of the standard PyTorch optimizers as from `torch.optim`, and all trainable parameters of the model should be of type `torch.nn.Parameter`. The variational parameters are predefined as part of the `VariationalGP` model.

In most cases, the boilerplate code below will work well. It has the same basic components as the standard PyTorch training loop:

    1. Zero all parameter gradients
    2. Call the model and compute the loss
    3. Call backward on the loss to fill in gradients
    4. Take a step on the optimizer

However, defining custom training loops allows for greater flexibility. For example, it is possible to learn the variational parameters and kernel hyperparameters with different learning rates.

```python
from gpytorch.mlls.variational_elbo import VariationalELBO

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
# num_data refers to the amount of training data
mll = VariationalELBO(likelihood, model, train_y.numel())

training_iter = 50
for i in range(training_iter):
    # Zero backpropped gradients from previous iteration
    optimizer.zero_grad()
    # Get predictive output
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iter, loss.item()))
    optimizer.step()
```

```output
Iter 1/50 - Loss: 1.225
Iter 2/50 - Loss: 8.492
Iter 3/50 - Loss: 2.531
Iter 4/50 - Loss: 3.006
Iter 5/50 - Loss: 5.014
Iter 6/50 - Loss: 3.859
Iter 7/50 - Loss: 1.783
Iter 8/50 - Loss: 1.525
Iter 9/50 - Loss: 2.158
Iter 10/50 - Loss: 2.525
Iter 11/50 - Loss: 2.080
Iter 12/50 - Loss: 1.602
Iter 13/50 - Loss: 1.520
Iter 14/50 - Loss: 1.704
Iter 15/50 - Loss: 1.773
Iter 16/50 - Loss: 1.597
Iter 17/50 - Loss: 1.438
Iter 18/50 - Loss: 1.574
Iter 19/50 - Loss: 1.795
Iter 20/50 - Loss: 1.737
Iter 21/50 - Loss: 1.847
Iter 22/50 - Loss: 1.789
Iter 23/50 - Loss: 1.505
Iter 24/50 - Loss: 1.369
Iter 25/50 - Loss: 1.503
Iter 26/50 - Loss: 1.363
Iter 27/50 - Loss: 1.322
Iter 28/50 - Loss: 1.330
Iter 29/50 - Loss: 1.378
Iter 30/50 - Loss: 1.343
Iter 31/50 - Loss: 1.416
Iter 32/50 - Loss: 1.467
Iter 33/50 - Loss: 1.441
Iter 34/50 - Loss: 1.425
Iter 35/50 - Loss: 1.327
Iter 36/50 - Loss: 1.498
Iter 37/50 - Loss: 1.393
Iter 38/50 - Loss: 1.208
Iter 39/50 - Loss: 1.429
Iter 40/50 - Loss: 1.361
Iter 41/50 - Loss: 1.435
Iter 42/50 - Loss: 1.287
Iter 43/50 - Loss: 1.673
Iter 44/50 - Loss: 1.601
Iter 45/50 - Loss: 1.275
Iter 46/50 - Loss: 1.321
Iter 47/50 - Loss: 1.750
Iter 48/50 - Loss: 1.487
Iter 49/50 - Loss: 1.195
Iter 50/50 - Loss: 1.430
```

**Make predictions with the model**

In the next cell, we make predictions with the model. To do this, we simply put the model and likelihood in eval mode, and call both modules on the test data.

In `.eval()` mode, when we call `model()` - we get GP’s latent posterior predictions. These will be `MultivariateNormal` distributions. But since we are performing binary classification, we want to transform these outputs to classification probabilities using our likelihood.

When we call `likelihood(model())`, we get a `torch.distributions.Bernoulli` distribution, which represents our posterior probability that the data points belong to the positive class.

```python
f_preds = model(test_x)
y_preds = likelihood(model(test_x))

f_mean = f_preds.mean
f_samples = f_preds.sample(sample_shape=torch.Size((1000,))
```

```python
# Go into eval mode
model.eval()
likelihood.eval()

with torch.no_grad():
    # Test x are regularly spaced by 0.01 0,1 inclusive
    test_x = torch.linspace(0, 1, 101)
    # Get classification predictions
    observed_pred = likelihood(model(test_x))

    # Initialize fig and axes for plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Get the predicted labels (probabilites of belonging to the positive class)
    # Transform these probabilities to be 0/1 labels
    pred_labels = observed_pred.mean.ge(0.5).float().mul(2).sub(1)
    ax.plot(test_x.numpy(), pred_labels.numpy(), 'b')
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
```
![Plot](https://gpytorch.readthedocs.io/en/latest/_images/examples_02_Simple_GP_Classification_Simple_GP_Classification_10_0.png)